### 1.c - Auditoria no Código Postal (Postal Code)

In [35]:
""" Auditoria nos códigos postais (CEP)

Realizar auditoria nos códigos postais com o objetivo de identificar caracteres especiais ou códigos fora do 
formato padrão

"""

import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint


# Arquivo XML que a ser auditado
OSMFILE = "dados/jundiai_e_regiao_map_zen.osm"

# Variáveis com padrões de expressões regulares
accept_pc1 = re.compile(r'(.....?-...)')
accept_pc2 = re.compile(r'(........)')
problemchars = re.compile(r'([=\+/&<>;\'"\?%#$@\,\.\t\r\n]|[A-z])')

postal_c = None 


def audit_characters_postal_code(element):
    """ Analisa códigos postais (CEP)
    
    Realizar análise dos códigos postais identificando caracteres especiais ou casos que estejam fora do formato
    padrão.
    
    Argumentos
        element (str): Código postal (CEP)
        
    Retorno:
        True se não tiver caracteres especiais e estiver dentro do padrdão esperado, False para os outros casos.
    
    """
    
    classificated = False
    result = False

    # INCOMPLETO
    if len(element.replace("-","")) < 8:
        result = False
        classificated = True        

    # PROBLEMCHAR
    if classificated == False:
        if problemchars.search(element) is not None:
            classificated = True        
        
    # CORRETO 2
    if not classificated:
        if accept_pc1.search(element) is not None:        
            result = True
            classificated = True   
            
    # CORRETO 1
    if not classificated:
        if accept_pc2.search(element) is not None:        
            result = False
            classificated = True   
    
    # OUTROS
    if not classificated:
        print(element, ' = classif.: OTHER')                  
    
    return result


def is_postal_code(elem):
    """ Verifica se é um código postal (CEP)
    
    Verifica o node e atributo para identificar se é um código postal
    
    Argumentos:
        elem (Element): Elemento com informações sobre o XML
    
    Retorno
        True se o elemento "k" for "addr:postcode" ou "postal_code"
    
    """
    return ((elem.attrib['k'] == "addr:postcode") | (elem.attrib['k'] == "postal_code"))


def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print("%s: %d" % (k, v))

        
def audit(osmfile):
    """ Realiza varredura no arquivo XML identifiando códigos postais e verificando se é válido.
        Caso inválido, realizar contágem e exibir na tela.
    """
    
    osm_file = open(osmfile, "r", encoding="utf-8")
    postal_codes = defaultdict(set)
    
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        #print('\n',elem.tag, elem.attrib)    
       
        if elem.tag == "node" or elem.tag == "way":
            #print('\n',elem.tag, elem.attrib)

            for tag in elem.iter("tag"):                
                if is_postal_code(tag): 
                    if audit_characters_postal_code(tag.attrib['v']) == False:
                        postal_c = tag.attrib['v']
                        if postal_codes[postal_c]== set():
                            postal_codes[postal_c] = 0

                        postal_codes[postal_c]+= 1
                        
                            
    print_sorted_dict(postal_codes)                
    osm_file.close()


def test():
    # Auditar para encontrar códigos postais (CEPs) fora do padrão
    st_types = audit(OSMFILE)
            

if __name__ == '__main__':
    test()
    

07600000: 129
12940700: 1
12942540: 1
12942655: 10
12943000: 2
12943310: 7
12943320: 13
12943330: 31
12943340: 38
12943350: 16
12943370: 13
12943380: 3
12943500: 12
12947452: 8
13203280: 1
13221550: 1
13240000: 6
3221-390: 1
